In [1]:
import os
import pyspark
import numpy as np
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark = SparkSession.builder.master("local[20]").appName("tc_dataprocessing.com").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

23/01/26 22:13:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
root = "/mnt/raid0_ssd_8tb/dataset/trace/"
files = os.listdir(root)
file_paths = [root + f for f in files]
# print(*file_paths, sep='\n')

In [4]:
len(file_paths)

211

In [5]:
df = spark.read.json(file_paths)

23/01/26 23:09:23 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
df.printSchema()

root
 |-- CDMVersion: string (nullable = true)
 |-- datum: struct (nullable = true)
 |    |-- com.bbn.tc.schema.avro.cdm18.EndMarker: struct (nullable = true)
 |    |    |-- recordCounts: struct (nullable = true)
 |    |    |    |-- EVENT_ACCEPT: string (nullable = true)
 |    |    |    |-- EVENT_CHANGE_PRINCIPAL: string (nullable = true)
 |    |    |    |-- EVENT_CLONE: string (nullable = true)
 |    |    |    |-- EVENT_CLOSE: string (nullable = true)
 |    |    |    |-- EVENT_CONNECT: string (nullable = true)
 |    |    |    |-- EVENT_CREATE_OBJECT: string (nullable = true)
 |    |    |    |-- EVENT_EXECUTE: string (nullable = true)
 |    |    |    |-- EVENT_EXIT: string (nullable = true)
 |    |    |    |-- EVENT_FORK: string (nullable = true)
 |    |    |    |-- EVENT_LINK: string (nullable = true)
 |    |    |    |-- EVENT_LOADLIBRARY: string (nullable = true)
 |    |    |    |-- EVENT_MMAP: string (nullable = true)
 |    |    |    |-- EVENT_MODIFY_FILE_ATTRIBUTES: string (nullabl

In [7]:
df = df.select("datum.*").select("`com.bbn.tc.schema.avro.cdm18.Subject`").withColumnRenamed("com.bbn.tc.schema.avro.cdm18.Subject", "subject").select("subject.*")

In [8]:
cmd_df = df.select("cmdLine.*").withColumnRenamed("string", "cmdLine")
cmd_df = cmd_df.na.drop()

In [9]:
cmd_df.count()

39283148

In [10]:
unique_cmd_df = cmd_df.select('cmdLine').distinct()

In [11]:
unique_cmd_df.count()

4194

In [12]:
unique_cmd_df

cmdLine
find /var/lib/apt...
/bin/sh /etc/upda...
ls /etc/bash_comp...
cat /var/lib/upda...
/usr/bin/python3 ...
cut 2D6420 -f4
/usr/lib/thunderb...
/bin/sh /usr/bin/...
sudo sysctl vm.di...
/usr/bin/clear_co...


In [13]:
unique_cmd_df.coalesce(1).write.format('json').save("/mnt/raid0_ssd_8tb/dataset/tc_cmdLine.df")